In [9]:
import os
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [10]:
nodes = pd.read_csv("../datasets/radnet_synthetic_nodes.csv", index_col=0)
edges = pd.read_csv("../datasets/radnet_synthetic_edges.csv")

# tensores
x = torch.tensor(nodes.values, dtype=torch.float)
edge_index = torch.tensor([edges["src"].values, edges["dst"].values], dtype=torch.long)

label_map = {"baixa": 0, "media": 1, "alta": 2}
y = torch.tensor([label_map[s] for s in edges["label"].values], dtype=torch.long)

In [11]:
test_size = 0.3        # fração para teste
random_state = 42      # semente para reprodutibilidade

all_idx = np.arange(edge_index.size(1))
train_idx, test_idx = train_test_split(
    all_idx,
    test_size=test_size,
    random_state=random_state,
    shuffle=True
)

train_idx = torch.tensor(train_idx, dtype=torch.long)
test_idx  = torch.tensor(test_idx,  dtype=torch.long)

train_edges = edge_index[:, train_idx]
test_edges  = edge_index[:, test_idx]
train_labels = y[train_idx]
test_labels  = y[test_idx]

In [12]:
class SimpleGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.fc1 = nn.Linear(in_channels, hidden_channels)
        self.fc2 = nn.Linear(hidden_channels, out_channels)
        
    def forward(self, x, adj):
        h = torch.matmul(adj, x)   # agregação
        h = F.relu(self.fc1(h))
        h = torch.matmul(adj, h)
        h = self.fc2(h)
        return h

class LinkPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes=3):
        super().__init__()
        self.lin1 = nn.Linear(in_channels * 2, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, num_classes)
        
    def forward(self, x_i, x_j):
        z = torch.cat([x_i, x_j], dim=-1)
        z = F.relu(self.lin1(z))
        return self.lin2(z)

In [13]:
num_nodes = x.size(0)
adj = torch.eye(num_nodes)
for i, j in zip(edges["src"], edges["dst"]):
    adj[i, j] = 1

# normalização
deg = adj.sum(1)
deg_inv = torch.pow(deg, -0.5)
deg_inv[deg_inv == float("inf")] = 0
D_inv = torch.diag(deg_inv)
adj_norm = D_inv @ adj @ D_inv

In [14]:
gcn = SimpleGCN(in_channels=x.size(1), hidden_channels=8, out_channels=4)
predictor = LinkPredictor(in_channels=4, hidden_channels=8, num_classes=3)
optimizer = torch.optim.Adam(list(gcn.parameters()) + list(predictor.parameters()), lr=0.01)

for epoch in range(1, 500):
    # treino
    gcn.train(); predictor.train()
    z = gcn(x, adj_norm)
    preds = []
    for i, j in zip(train_edges[0], train_edges[1]):
        preds.append(predictor(z[i].unsqueeze(0), z[j].unsqueeze(0)))
    preds = torch.cat(preds, dim=0)
    
    loss = F.cross_entropy(preds, train_labels)
    optimizer.zero_grad(); loss.backward(); optimizer.step()
    
    # teste periódico
    if epoch % 10 == 0:
        gcn.eval(); predictor.eval()
        z = gcn(x, adj_norm)
        preds = []
        for i, j in zip(test_edges[0], test_edges[1]):
            preds.append(predictor(z[i].unsqueeze(0), z[j].unsqueeze(0)))
        preds = torch.cat(preds, dim=0)
        
        y_true = test_labels.numpy()
        y_pred = preds.argmax(dim=-1).detach().numpy()
        acc = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average="macro")
        
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f} | Test Acc: {acc:.4f} | Test F1: {f1:.4f}")

Epoch 010 | Loss: 1.0724 | Test Acc: 0.4000 | Test F1: 0.1905
Epoch 020 | Loss: 0.8943 | Test Acc: 0.2000 | Test F1: 0.1333
Epoch 030 | Loss: 0.7152 | Test Acc: 0.4000 | Test F1: 0.2222
Epoch 040 | Loss: 0.5252 | Test Acc: 0.2000 | Test F1: 0.2222
Epoch 050 | Loss: 0.3366 | Test Acc: 0.2000 | Test F1: 0.2222
Epoch 060 | Loss: 0.1489 | Test Acc: 0.4000 | Test F1: 0.3556
Epoch 070 | Loss: 0.0395 | Test Acc: 0.4000 | Test F1: 0.3556
Epoch 080 | Loss: 0.0098 | Test Acc: 0.4000 | Test F1: 0.3556
Epoch 090 | Loss: 0.0037 | Test Acc: 0.4000 | Test F1: 0.3556
Epoch 100 | Loss: 0.0020 | Test Acc: 0.4000 | Test F1: 0.3556
Epoch 110 | Loss: 0.0014 | Test Acc: 0.4000 | Test F1: 0.3556
Epoch 120 | Loss: 0.0012 | Test Acc: 0.4000 | Test F1: 0.3556
Epoch 130 | Loss: 0.0010 | Test Acc: 0.4000 | Test F1: 0.3556
Epoch 140 | Loss: 0.0009 | Test Acc: 0.4000 | Test F1: 0.3556
Epoch 150 | Loss: 0.0008 | Test Acc: 0.4000 | Test F1: 0.3556
Epoch 160 | Loss: 0.0007 | Test Acc: 0.4000 | Test F1: 0.3556
Epoch 17

In [15]:
os.makedirs("../models", exist_ok=True)

ckpt_gcn = "../models/gcn_state.pt"
ckpt_pred = "../models/predictor_state.pt"
ckpt_meta = "../models/meta.pt"
ckpt_graph = "../models/graph_artifacts.pt"

# Salva pesos
torch.save(gcn.state_dict(), ckpt_gcn)
torch.save(predictor.state_dict(), ckpt_pred)

# Salva metadados
torch.save({
    "num_features": x.size(1),   # numero de features do nó (aqui 3)
    "gcn_hidden": 8,             # tamanho da hidden layer do GCN
    "gcn_out": 4,                # saída do GCN
    "pred_hidden": 8,            # hidden layer do LinkPredictor
    "num_classes": 3             # numero de classes (baixa, media, alta)
}, ckpt_meta)

# Salva artefatos do grafo
torch.save({
    "x": x.cpu(),              # features dos nós
    "adj_norm": adj_norm.cpu() # adjacência normalizada
}, ckpt_graph)

print("Modelo salvo em ../models/gcn_state.json")
print("Modelo salvo em ../models/predictor_state.json")
print("Modelo salvo em ../models/meta.json")
print("Modelo salvo em ../models/graph_artifacts.json")

Modelo salvo em ../models/gcn_state.json
Modelo salvo em ../models/predictor_state.json
Modelo salvo em ../models/meta.json
Modelo salvo em ../models/graph_artifacts.json


In [16]:
class_names = ["baixa", "media", "alta"]  # nomes de classes

gcn.eval(); predictor.eval()
with torch.no_grad():
    # embeddings dos nós
    z = gcn(x, adj_norm)

    def eval_split(edges_tensor, labels_tensor):
        # logits para cada aresta do split
        logits = []
        for i, j in zip(edges_tensor[0], edges_tensor[1]):
            logits.append(predictor(z[i].unsqueeze(0), z[j].unsqueeze(0)))
        logits = torch.cat(logits, dim=0)

        y_true = labels_tensor.cpu().numpy()
        y_pred = logits.argmax(dim=-1).cpu().numpy()

        # métricas agregadas
        acc = accuracy_score(y_true, y_pred)
        prec_macro  = precision_score(y_true, y_pred, average="macro",  zero_division=0)
        rec_macro   = recall_score(y_true,    y_pred, average="macro",  zero_division=0)
        f1_macro    = f1_score(y_true,        y_pred, average="macro",  zero_division=0)

        prec_weight = precision_score(y_true, y_pred, average="weighted", zero_division=0)
        rec_weight  = recall_score(y_true,    y_pred, average="weighted", zero_division=0)
        f1_weight   = f1_score(y_true,        y_pred, average="weighted", zero_division=0)

        # métricas por classe (mesma ordem de class_names)
        prec_per_class = precision_score(y_true, y_pred, average=None, zero_division=0)
        rec_per_class  = recall_score(y_true,    y_pred, average=None, zero_division=0)
        f1_per_class   = f1_score(y_true,        y_pred, average=None, zero_division=0)

        per_class = {
            cname: {
                "precision": float(prec_per_class[idx]),
                "recall":    float(rec_per_class[idx]),
                "f1":        float(f1_per_class[idx]),
            }
            for idx, cname in enumerate(class_names)
        }

        aggregate = {
            "accuracy": float(acc),
            "precision_macro":  float(prec_macro),
            "recall_macro":     float(rec_macro),
            "f1_macro":         float(f1_macro),
            "precision_weighted": float(prec_weight),
            "recall_weighted":    float(rec_weight),
            "f1_weighted":        float(f1_weight),
        }

        return aggregate, per_class

    train_agg, train_per_class = eval_split(train_edges, train_labels)
    test_agg,  test_per_class  = eval_split(test_edges,  test_labels)

# Monta dicionário e salva em JSON
results = {
    "task": "edge_load_classification",
    "classes": class_names,
    "train": {
        **train_agg,
        "per_class": train_per_class,
    },
    "test": {
        **test_agg,
        "per_class": test_per_class,
    }
}

os.makedirs("../results", exist_ok=True)
with open("../results/result.json", "w") as f:
    json.dump(results, f, indent=2)

print("Métricas salvas em ../results/result.json")

Métricas salvas em ../results/result.json
